# GPT to Llama Implementation

Implementing the conversion from GPT architecture to Llama models using template by Sebastian Rascha and "Building a Large Language Model from Scratch." Based on the context from the standalone Llama 3.2 notebook, this implementation would include the key architectural differences between GPT and Llama models, such as the transition from standard multi-head attention to Grouped Query Attention (GQA), the implementation of RoPE (Rotary Position Embedding) instead of absolute positional encodings, the use of SwiGLU activation functions in the feed-forward networks, and RMSNorm instead of LayerNorm for normalization.

The implementation serves as a practical guide for understanding how modern language models like Llama 3.2 evolved from the original GPT architecture, providing hands-on code examples of each component transformation. This would be particularly valuable for researchers and developers looking to understand the technical improvements that make Llama models more efficient and performant compared to earlier. 

In [10]:
os.chdir("/Users/anup/gitProjects/language-models-psychiatry")
print("Changed working directory to:", os.getcwd())


Changed working directory to: /Users/anup/gitProjects/language-models-psychiatry


In [11]:
# Install packages individually for better error handling
packages = [
    "blobfile",
    "huggingface_hub",
    "ipywidgets",
    "safetensors",
    "sentencepiece"
]

for package in packages:
    %pip install {package}

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [12]:
from importlib.metadata import version

pkgs = [
    "blobfile",         #required to download pretrained weights
    "huggingface_hub",   #required to download pretrained weights
    "tiktoken",          #required to implement the tokenizer
    "torch",             #required to implement models
]

for packages in pkgs:
    print(f"{packages} version: {version(packages)}")

blobfile version: 3.1.0
huggingface_hub version: 0.27.1
tiktoken version: 0.11.0
torch version: 2.8.0
